# Tarea 1, High Performance Computing 2018-2

Con el fin de realizar un repaso de las habilidades de programación, se debe construir un programa que permita multiplicar 2 matrices, teniendo en cuenta las siguientes condiciones :

* El programa debe permite generar matrices aleatorias.
* Se deben realizar las implementaciones : 
    - Secuencial
    - Acelerada 
        - Hilos vs Procesos 
* Se deben construir graficas de tiempo vs tamaño de la matriz. 
    - Los tiempos deben ser promediados


El resulta de la multiplicación de las dos matrices debe ser enviado via stdout aun archivo de texto,
utilizar ***[HPCG1]*** es el asunto que debe tener el email enviado al docente Ramiro.

<cite>Como tipo de dato se utiliza long long ***int64***</cite>

# Solución

## Multiplicación de Matrices
dadas dos matrices $A$,$B$ de la forma:
$$A = \begin{pmatrix}
 a_{1 1} & \cdots & a_{1 n} \\
 \vdots & \ddots & \vdots \\
 a_{m 1} & \cdots & a_{m n}
 \end{pmatrix}, B = \begin{pmatrix}
 b_{1 1} & \cdots & b_{1 n} \\
 \vdots & \ddots & \vdots \\
 b_{m 1} & \cdots & b_{m n}
 \end{pmatrix}
 $$

Escritas en los textos como $A:=(a_{i j})_{m \times n}$, $B:=(b_{i j})_{n \times p}$, donde $m,n,p$ indican las filas y columnas de cada matriz, el producto de $A\cdot B$ es:
$C = AB_{}^{}$.

$$ \begin{pmatrix}
 a_{1 1} & \cdots & a_{1 n} \\
 \vdots & \ddots & \vdots \\
 a_{m 1} & \cdots & a_{m n}
 \end{pmatrix} \cdot \begin{pmatrix}
 b_{1 1} & \cdots & b_{1 p} \\
 \vdots & \ddots & \vdots \\
 b_{n 1} & \cdots & b_{n p}
 \end{pmatrix}$$
 
 $$\begin{pmatrix}
 a_{11}b_{11}+ \cdots +a_{1n}b_{n1} & \cdots & a_{11}b_{1p}+ \cdots +a_{1n}b_{np} \\
 \vdots & \ddots & \vdots \\
 a_{m1}b_{11}+ \cdots +a_{mn}b_{n1} & \cdots & a_{m1}b_{1p}+ \cdots +a_{mn}b_{np}
 \end{pmatrix}
$$

Que no es mas que la sumatoria de multiplicar la fila por la columna para cada elemento de la matriz resulta:
$$c_{ij} = \sum_{r=1}^n a_{ir}b_{rj}$$


*Nota[!]:La cantindad de Columnas debe ser igual a la cantidad de filas de la segunda matriz,$A:=(a_{i j})_{m \times n}$, $B:=(b_{i j})_{n \times p}$, tendra como resultado una Matriz $B:=(b_{i j})_{m \times p}$*.

De las observaciones anteriores podemos decir que  para resolver la multiplicación de las matrices $A\cdot B$ es necesario realizar $m*n*p$ multiplicaciones, Seria posible utilizar una gran cantidad de tecnicas como se describe  en[[1][1]]y [[2][2]], pero por tiempo utilizare la version interactiva tiene un costo $Θ(n^{3})$ como se muestra en [[3][3]]:
<img src="http://i.imgur.com/Y4OmGFt.png" height="650" width="640">

[1]: https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm#Iterative_algorithm
[2]: https://es.wikibooks.org/wiki/Optimizaci%C3%B3n_del_Producto_de_Matrices
[3]: https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm#Iterative_algorithm

## Implementación práctica

Para realizar la implementacion de la tarea he decidido usar Python en su version 3.6, por facilidad y para un facil manejo del notebook he construido una clase ***Matriz*** que posee los metodos : 
* **naiveProdSeq()** : Ejecucion de una multiplicacion de matrices de manera secuencial. 
* **parallelNaiveProduct()**: Ejecucion de una multiplicacion de matrices utilizando multithreading
* **processNaiveProduct()**: Ejecución de una multiplicación de matrice utizando multiprocessing. 

Además se intenta realizar la optimización del algoritmo para bajar la complejidad anteriormente mencionada, se realiza la multiplicación utilizando numpy que evita el problema del [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) del cual hablaremos mas adelante.


In [1]:
#!/usr/bin/python
# encoding=utf8
# Developer: hfjimenez@utp.edu.co, Homework #1
# High Performance Computing, UTP, 2018-2
# Notes : Conformable matrix, Acols == Brows.
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import numpy as np
import time
from pprint import pprint
from src.util import *


class MatrizError(Exception):
    """ An exception class for Matrix """
    pass


class Matriz(object):
    """ A matriz representation """

    def __init__(self, rows, cols, prng=True, maxr=100):
        print('{}Generando Matriz'.format(ok))
        if prng:
            self.rows = rows
            self.cols = cols
            self.m = np.random.randint(0, maxr, size=(
                self.rows, self.cols), dtype=np.int64)
            print('{}Matriz Aleatoria creada'.format(ok))
        else:
            self.m = np.zeros((rows, cols), dtype=np.int64)
            # self.m = [[0]*n for x in range(m)]    # Another way if I don't want to use np

    def info(self):
        print('Matriz Information:\nDimensions:{}\nSize:{}\nShape:{}\nDatatype in usage:{}\nItemSize:{}bytes'.format(
            self.m.ndim, self.m.size, self.m.shape, self.m.dtype, self.m.itemsize))

    def printm(self):
        pprint(self.m)

    def naiveProdSeq(self, B):
        """ n**3 multiplication, sequential """
        if not (self.cols == B.rows):
            raise MatrizError(
                "Error, numero de columnas de A debe ser igual al numero de Filas de B!")
        result = np.zeros((self.rows, B.cols), dtype=np.int64)
        tm = len(self.m)    #Evitar el llamado a 10101 funciones de len.
        tb = len(B.m)
        for row in range(tm):
            for col in range(tb):
                for k in range(tm):
                    result[row][col] += self.m[row][k] * B.m[k][col]
        return result
    
    def naiveProdSeq2(self, B):
        """ n**3 multiplication, sequential """
        if not (self.cols == B.rows):
            raise MatrizError(
                "Error, numero de columnas de A debe ser igual al numero de Filas de B!")
        result = np.zeros((self.rows, B.cols), dtype=np.int64)
        for row in range(len(self.m)):
            for k in range(len(self.m)):
                for col in range(len(B.m)):
                    result[row][col] += self.m[row][k] * B.m[k][col]
        return result
    def numpyProduct(self, B):
        """My simple wrapper to multiply two matrix"""
        result = np.multiply(self.m,B.m) # Fast Multiplication.
        return result
        

*Notese* que para la construcción inicial de la matriz, se utiliza como representación los arrays de numpy, debido a que es mucho mas conveniente por eficienca y manejabilidad [4][4] y por complejidad espacial.


[4]: https://stackoverflow.com/questions/993984/why-numpy-instead-of-python-lists


In [2]:
#Para demostrar el facil uso de esta clase crearemos 2 matrices de 100*100
A = Matriz(100,100)
# Si la imprimimos obtenemos :
A.printm()

[✓] Generando Matriz
[✓] Matriz Aleatoria creada
array([[63, 91, 30, ..., 38, 75, 38],
       [80, 80, 93, ..., 20,  9,  6],
       [26, 33, 78, ..., 16, 25, 43],
       ..., 
       [54, 74, 20, ..., 51, 55, 51],
       [66, 22, 50, ..., 39, 11, 63],
       [41, 64,  4, ...,  4, 92, 52]])


In [3]:
B = Matriz(100,100)
B.printm()

[✓] Generando Matriz
[✓] Matriz Aleatoria creada
array([[64, 50, 81, ..., 90, 92, 72],
       [31, 40, 91, ..., 53, 72, 38],
       [11, 15, 87, ..., 28, 80, 50],
       ..., 
       [12, 14, 99, ..., 82, 14, 26],
       [16, 96, 45, ..., 40,  3,  1],
       [95, 39, 57, ..., 48, 82, 72]])


In [4]:
# Si deseamos producir A*B, estadisticamente obtener el promedio con 3 veces de ejecucion 
%timeit -n 3 A.naiveProdSeq(B)  

1.25 s ± 93.6 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [5]:
# Sera que si tomamos cols primero, afectara el resultado ? de la forma ijk a ikj ?
%timeit -n 3 A.naiveProdSeq2(B)

1.24 s ± 129 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [6]:
%timeit -n 3 A.numpyProduct(B)

The slowest run took 8.45 times longer than the fastest. This could mean that an intermediate result is being cached.
32.4 µs ± 31.2 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [7]:
# Es extraño el resultado de arriba, por que numpy corre mas rapido
# Es posible que numpy acelere la ejecucion solo por si las moscas ejecutemos de nuevo perocon 5 veces : 
%timeit -n 3 A.numpyProduct(B) # A*B, con numpy
%timeit -n 3 B.numpyProduct(A) # B*A, con numpy


39 µs ± 23.6 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)
The slowest run took 8.26 times longer than the fastest. This could mean that an intermediate result is being cached.
47.4 µs ± 58 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [8]:
# Es posible saber en que parte del codigo es que se consume el mayor tiempo, 
# usando el profiler de python : 

%prun  B.numpyProduct(A)  


In [9]:
# Ahora si  usamos el mismo profiler con nuestra multiplicacion ingenua : 
%prun A.naiveProdSeq(B) 

Ahora si queremos acelerar este procedimiento de la multiplicacion ingenua de matrices, podriamos usar algunas de las dos opciones  que hay : 
- Creacion de Multiples Hilos, pool de hilos(***Threads:Multithreading***)
- Creacion de Multiples Procesos, pool de procesos (***Process:Multiprocessing***)

En python3.6 contamos con varias librerias que permiteno hacer esto .

#### Referencias :
- https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm#Iterative_algorithm
- https://es.wikibooks.org/wiki/Optimizaci%C3%B3n_del_Producto_de_Matrices
- https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm#Iterative_algorithm
- https://stackoverflow.com/questions/993984/why-numpy-instead-of-python-lists
- https://martin-thoma.com/matrix-multiplication-python-java-cpp/